In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.34.0 trl==0.4.7 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 19.8 MB/s eta 0:00:00


In [ ]:
%%time

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    LlamaForCausalLM
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import sentencepiece
import huggingface_hub

CPU times: user 8.39 s, sys: 1.37 s, total: 9.76 s
Wall time: 15.7 s


In [ ]:
%%time

#Model, Dataset, & NewModel
# CurModel = "mistralai/Mistral-7B-v0.1"
CurModel = "NousResearch/Llama-2-7b-hf"
CurDataset = "Domwerr/EmojiDataset" #EmojiToSentence
# CurDataset = "Domwerr/EmojiToResponseDataset" #EmojiToResponse
NextModel = "llama2-7b-EmojiToResponse-epoch4-2e-4-test"
#=========================================================================================================================================
# QLoRA parameters
# LoRA attention dimension
LoraAtteDim = 64
# Alpha parameter
LoraA = 16
# Dropout probability
LoraDropoutProb = 0.1

#=========================================================================================================================================
# bitsandbytes parameters
# Use4Bits?
Using4Bits = True
# Compute dtype
ComputeDtype = "float16"
# Quantization type[fp4/nf4]
QuantType = "nf4"
# Nested quantization?
UsingNestedQuant = False

#=========================================================================================================================================
# Training Arguments Parameter
# Output directory
OutputDir = "./results"
# Number Of Epochs
TotalEpoch = 4
# Batch Size/GPU for Training & Evaluation
BatchSizeTraining = 4
BatchSizeEval = 1
# Steps to Accumulate Gradient
GradAccumulationSteps = 1
# Maximum gradient normal
MaxGradientNormal = 0.3
# LearningRate
LearningRate = 2e-4
# Weight Decay
WeightDecay = 0.001
# Optimizer
OptimizerAlgo = "paged_adamw_32bit"
# Learning Rate Scheduler
LR_Scheduler = "constant"
# TrainingSteps (If not used Epoch)
# max_steps = 500
# Linear Warmup Ratio
WarmupRatio = 0.03
# Group the Sequences by Length (To Batches) (Good for Efficiency)
GroupByLength = True
# Save Checkpoints (If using Steps)
SavingSteps = 100
# Log Updates (if Using Steps)
LoggingSteps = 100

#=========================================================================================================================================
# SFT parameters
# Maximum Sequence Length
MaxSequenceLength = 2048
# Packing Some Shorts Example for efficiency
PackMultipleShort = False
# Load Model to Target GPU
DeviceMap = {"": 0}

# Load Dataset
dataset = load_dataset(CurDataset, split="train")
dataset2 = load_dataset(CurDataset, split="validation")


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

CPU times: user 1.71 s, sys: 64.9 ms, total: 1.78 s
Wall time: 17 s


In [ ]:
%%time

ComputeDtypeAttr = getattr(torch, ComputeDtype)

BNBConfig = BitsAndBytesConfig(
    load_in_4bit=Using4Bits,
    bnb_4bit_quant_type=QuantType,
    bnb_4bit_compute_dtype=ComputeDtypeAttr,
    bnb_4bit_use_double_quant=UsingNestedQuant,
)

# Check GPU compatibility with bfloat16
if ComputeDtypeAttr == torch.float16 and Using4Bits:
    OutputMajor, _ = torch.cuda.get_device_capability()
    if OutputMajor >= 8:
        print("=" * 80)
        print("Supports BF16")
        print("=" * 80)

# LoadModel
LoadModel = AutoModelForCausalLM.from_pretrained(
    CurModel,
    quantization_config=BNBConfig,
    device_map=DeviceMap
)
LoadModel.config.use_cache = False
LoadModel.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(CurModel, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
PEFTConfig = LoraConfig(
    lora_alpha=LoraA,
    lora_dropout=LoraDropoutProb,
    r=LoraAtteDim,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
TraiingArgumentsParam = TrainingArguments(
    output_dir=OutputDir,
    num_train_epochs=TotalEpoch,
    per_device_train_batch_size=BatchSizeTraining,
    gradient_accumulation_steps=GradAccumulationSteps,
    max_grad_norm=MaxGradientNormal,
    optim=OptimizerAlgo,
    learning_rate=LearningRate,
    lr_scheduler_type=LR_Scheduler,
    weight_decay=WeightDecay,
    warmup_ratio=WarmupRatio,
    group_by_length=GroupByLength,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_steps=SavingSteps,
    logging_steps=LoggingSteps,
    fp16=False,
    bf16=False,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=LoadModel,
    train_dataset=dataset,
    eval_dataset=dataset2,
    peft_config=PEFTConfig,
    dataset_text_field="prompt_and_question",
    max_seq_length=MaxSequenceLength,
    tokenizer=tokenizer,
    args=TraiingArgumentsParam,
    packing=PackMultipleShort,
)

print("START TRAINING")
# Train model
trainer.train()
print("FINISH TRAINING")
# Save trained model
trainer.model.save_pretrained(NextModel)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/893 [00:00<?, ? examples/s]

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

START TRAINING


You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.067700,1.016306
2,0.966600,0.954075
3,0.837800,0.956926
4,0.759200,0.974730


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

FINISH TRAINING
CPU times: user 14min 46s, sys: 8min 18s, total: 23min 5s
Wall time: 33min 51s


In [ ]:
%%time
#Use Personal Token
huggingface_hub.login(token = "")


# CurModel = "mistralai/Mistral-7B-v0.1"
CurModel = "NousResearch/Llama-2-7b-hf"
NextModel = "llama2-7b-EmojiToResponse-epoch4-2e-4"
#=========================================================================================================================================
# QLoRA parameters
# LoRA attention dimension
LoraAtteDim = 64
# Alpha parameter
LoraA = 16
# Dropout probability
LoraDropoutProb = 0.1

#=========================================================================================================================================
# bitsandbytes parameters
# Use4Bits?
Using4Bits = True
# Compute dtype
ComputeDtype = "float16"
# Quantization type[fp4/nf4]
QuantType = "nf4"
# Nested quantization?
UsingNestedQuant = False

#=========================================================================================================================================
# Training Arguments Parameter
# Output directory
OutputDir = "./results"
# Number Of Epochs
TotalEpoch = 4
# Batch Size/GPU for Training & Evaluation
BatchSizeTraining = 4
BatchSizeEval = 1
# Steps to Accumulate Gradient
GradAccumulationSteps = 1
# Maximum gradient normal
MaxGradientNormal = 0.3
# LearningRate
LearningRate = 2e-4
# Weight Decay
WeightDecay = 0.001
# Optimizer
OptimizerAlgo = "paged_adamw_32bit"
# Learning Rate Scheduler
LR_Scheduler = "constant"
# TrainingSteps (If not used Epoch)
# max_steps = 500
# Linear Warmup Ratio
WarmupRatio = 0.03
# Group the Sequences by Length (To Batches) (Good for Efficiency)
GroupByLength = True
# Save Checkpoints (If using Steps)
SavingSteps = 100
# Log Updates (if Using Steps)
LoggingSteps = 100

#=========================================================================================================================================
# SFT parameters
# Maximum Sequence Length
MaxSequenceLength = 2048
# Packing Some Shorts Example for efficiency
PackMultipleShort = False
# Load Model to Target GPU
DeviceMap = {"": 0}

BaseModel = AutoModelForCausalLM.from_pretrained(
    CurModel,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=DeviceMap,
)
MergedModel = PeftModel.from_pretrained(BaseModel, NextModel)
MergedModel = MergedModel.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(CurModel, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

NextModel = "llama2-7b-EmojiToResponse-epoch4-2e-4"
MergedModel.push_to_hub(NextModel, max_shard_size = "4000MB",use_temp_dir=False)
tokenizer.push_to_hub(NextModel, max_shard_size = "4000MB", use_temp_dir=False)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

...

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CPU times: user 1min 47s, sys: 1min 5s, total: 2min 53s
Wall time: 6min 50s


CommitInfo(commit_url='https://huggingface.co/Domwerr/llama2-7b-EmojiToResponse-epoch4-2e-4/commit/f85ba119a1307b57b07e6b8f8a9d4a419a5a8058', commit_message='Upload tokenizer', commit_description='', oid='f85ba119a1307b57b07e6b8f8a9d4a419a5a8058', pr_url=None, pr_revision=None, pr_num=None)